# Querying XML in SQL Server – Part 7 – Creating XML

[Last time](https://barneylawrence.com/2021/04/28/querying-xml-in-sql-server-part-6-sequencing-elements-and-finding-positions/) I promised to start looking at performance and XML indexes. I’ll get there but I realised I needed a decent test data set and couldn’t easily lay my hands on one that matched my needs so had to start building one myself. Part way through the process I figured I may as well squeeze an extra post out of the process and blog the creation process.

In my experience creating XML is a less common task than reading it with one semi-exception (that being the old trick of using FOR XML PATH(”) to concatenate strings) but it can be an important one. The biggest piece of XML I’ve built was for a [government mandated return](https://www.gov.uk/guidance/children-in-need-census), it was the only XML I built on that job but a sizeable proportion of the entire department’s work culminated in it. The switch from a set of Excel macros which took over eight hours to run to something that could rebuilt in minutes had a huge impact on our efficiency, particularly when there would be multiple rounds of validation failures against initial attempts at submission.

There are a few variants when it comes to converting queries into XML but in my opinion the only one you really need to know is the XML PATH approach, the others being either [fiddly](https://docs.microsoft.com/en-us/sql/relational-databases/xml/use-explicit-mode-with-for-xml?view=sql-server-ver15) or [not providing enough control](https://docs.microsoft.com/en-us/sql/relational-databases/xml/use-auto-mode-with-for-xml?view=sql-server-ver15).

Because all the cool kids are using it and I had a copy available from his recent live steamed courses I’m going to use a copy of the Stack Overflow database that Brent Ozar maintains. You can get your [own copy here](https://www.brentozar.com/archive/2015/10/how-to-download-the-stack-overflow-database-via-bittorrent/). I’m using the 2013 version but if you want to follow along any variant should work here. I’ve used TOP to limit results returned so that they’re short enough to place in the Blog.

## How Do I Build XML In SQL Server?

At the simplest level we just drop FOR XML PATH after a query, we give a value to the path which becomes the element each row is nested in and give a root value that wraps the whole output.

In [2]:
USE StackOverflow2013;

SELECT TOP 10
    U.Id AS 'UserId',
    U.DisplayName,
    U.UpVotes,
    U.DownVotes
FROM dbo.Users AS U
FOR XML PATH('User'), ROOT('Users');

(10 rows affected)

Total execution time: 00:00:00.106

XML_F52E2B61-18A1-11d1-B105-00805F49916B
<Users><User><UserId>17</UserId><DisplayName>Nick Berardi</DisplayName><UpVotes>884</UpVotes><DownVotes>215</DownVotes></User><User><UserId>19</UserId><DisplayName>Mads Kristiansen</DisplayName><UpVotes>36</UpVotes><DownVotes>12</DownVotes></User><User><UserId>20</UserId><DisplayName>Tom</DisplayName><UpVotes>370</UpVotes><DownVotes>38</DownVotes></User><User><UserId>22</UserId><DisplayName>Matt MacLean</DisplayName><UpVotes>202</UpVotes><DownVotes>5</DownVotes></User><User><UserId>23</UserId><DisplayName>Jax</DisplayName><UpVotes>571</UpVotes><DownVotes>7</DownVotes></User><User><UserId>24</UserId><DisplayName>sanmiguel</DisplayName><UpVotes>537</UpVotes><DownVotes>3</DownVotes></User><User><UserId>25</UserId><DisplayName>CodingWithoutComments</DisplayName><UpVotes>539</UpVotes><DownVotes>83</DownVotes></User><User><UserId>26</UserId><DisplayName>Shawn</DisplayName><UpVotes>565</UpVotes><DownVotes>67</DownVotes></User><User><UserId>27</UserId><DisplayName>denny</DisplayName><UpVotes>372</UpVotes><DownVotes>3</DownVotes></User><User><UserId>29</UserId><DisplayName>Michael Haren</DisplayName><UpVotes>5782</UpVotes><DownVotes>179</DownVotes></User></Users>


<span style="color: rgb(26, 26, 26); font-family: Merriweather, Georgia, serif; font-size: 16px; background-color: rgb(255, 255, 255);">Which gets something like this (this is going to be a post where the SQL notebook version and its embedded query results is going to be really helpful):</span>

<table border="0" cellpadding="0" cellspacing="0" style="border-collapse: separate; width: 600px; box-sizing: content-box !important; border-width: 0px !important; border-style: initial !important; margin: 0px !important; border-radius: 0px !important; background: none !important; float: none !important; height: auto !important; line-height: 1.1em !important; outline: 0px !important; overflow: visible !important; padding: 0px !important; position: static !important; vertical-align: baseline !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important;"><tbody style="box-sizing: content-box !important; border-radius: 0px !important; background: none !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important;"><tr style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-color: initial !important; border-width: 0px !important; border-style: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important;"><td class="gutter" style="box-sizing: content-box !important; border-width: 0px !important; border-style: initial !important; padding: 0px !important; border-radius: 0px !important; background: none !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; color: rgb(175, 175, 175) !important;"><div class="line number1 index0 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">1</div><div class="line number2 index1 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">2</div><div class="line number3 index2 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">3</div><div class="line number4 index3 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">4</div><div class="line number5 index4 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">5</div><div class="line number6 index5 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">6</div><div class="line number7 index6 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">7</div><div class="line number8 index7 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">8</div><div class="line number9 index8 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">9</div><div class="line number10 index9 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">10</div><div class="line number11 index10 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">11</div><div class="line number12 index11 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">12</div><div class="line number13 index12 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">13</div><div class="line number14 index13 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">14</div></td><td class="code" style="width: 555.333px; box-sizing: content-box !important; border-width: 0px !important; border-style: initial !important; padding: 0px !important; border-radius: 0px !important; background: none !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important;"><div class="container" style="box-sizing: content-box !important; border-radius: 0px !important; background: none !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px !important; position: relative !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important;"><div class="line number1 index0 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Users</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number2 index1 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">User</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number3 index2 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">UserId</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;17&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">UserId</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number4 index3 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DisplayName</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;Nick Berardi&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DisplayName</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number5 index4 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">UpVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;884&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">UpVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number6 index5 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DownVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;215&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DownVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number7 index6 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">User</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number8 index7 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">User</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number9 index8 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">UserId</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;19&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">UserId</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number10 index9 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DisplayName</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;Mads Kristiansen&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DisplayName</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number11 index10 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">UpVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;36&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">UpVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number12 index11 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DownVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;12&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DownVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number13 index12 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">User</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number14 index13 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Users</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div></div></td></tr></tbody></table>

## How do I Create Attributes In XML With SQL Server?

Adding Attributes is easy, we just give the relevant column a name commencing with an @ symbol.

In [3]:
SELECT TOP 2
    U.Id AS '@UserId',
    U.DisplayName,
    U.UpVotes,
    U.DownVotes
FROM dbo.Users AS U
FOR XML PATH('User'), ROOT('Users');

(2 rows affected)

Total execution time: 00:00:00.016

XML_F52E2B61-18A1-11d1-B105-00805F49916B
"<Users><User UserId=""17""><DisplayName>Nick Berardi</DisplayName><UpVotes>884</UpVotes><DownVotes>215</DownVotes></User><User UserId=""19""><DisplayName>Mads Kristiansen</DisplayName><UpVotes>36</UpVotes><DownVotes>12</DownVotes></User></Users>"


And now the XML looks like this:

<table border="0" cellpadding="0" cellspacing="0" style="border-collapse: separate; width: 600px; box-sizing: content-box !important; border-width: 0px !important; border-style: initial !important; margin: 0px !important; border-radius: 0px !important; background: none !important; float: none !important; height: auto !important; line-height: 1.1em !important; outline: 0px !important; overflow: visible !important; padding: 0px !important; position: static !important; vertical-align: baseline !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important;"><tbody style="box-sizing: content-box !important; border-radius: 0px !important; background: none !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important;"><tr style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-color: initial !important; border-width: 0px !important; border-style: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important;"><td class="gutter" style="box-sizing: content-box !important; border-width: 0px !important; border-style: initial !important; padding: 0px !important; border-radius: 0px !important; background: none !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; color: rgb(175, 175, 175) !important;"><div class="line number1 index0 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">1</div><div class="line number2 index1 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">2</div><div class="line number3 index2 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">3</div><div class="line number4 index3 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">4</div><div class="line number5 index4 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">5</div><div class="line number6 index5 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">6</div><div class="line number7 index6 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">7</div><div class="line number8 index7 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">8</div><div class="line number9 index8 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">9</div><div class="line number10 index9 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">10</div><div class="line number11 index10 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">11</div><div class="line number12 index11 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">12</div></td><td class="code" style="width: 555.333px; box-sizing: content-box !important; border-width: 0px !important; border-style: initial !important; padding: 0px !important; border-radius: 0px !important; background: none !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important;"><div class="container" style="box-sizing: content-box !important; border-radius: 0px !important; background: none !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px !important; position: relative !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important;"><div class="line number1 index0 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Users</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number2 index1 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">User</code> <code class="xml color1" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: gray !important;">UserId</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">=</code><code class="xml string" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: blue !important;">"17"</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number3 index2 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DisplayName</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;Nick Berardi&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DisplayName</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number4 index3 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">UpVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;884&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">UpVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number5 index4 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DownVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;215&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DownVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number6 index5 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">User</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number7 index6 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">User</code> <code class="xml color1" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: gray !important;">UserId</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">=</code><code class="xml string" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: blue !important;">"19"</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number8 index7 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DisplayName</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;Mads Kristiansen&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DisplayName</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number9 index8 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">UpVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;36&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">UpVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number10 index9 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DownVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;12&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DownVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number11 index10 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">User</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number12 index11 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Users</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div></div></td></tr></tbody></table>

## How Do I Nest Elements In XML With SQL Server?

If we want to start grouping elements under parent tags then we just name the columns with the needed path. If two consecutive fields have a common element to the path they get grouped together. Here we nest up and down votes in a votes element.

In [4]:
SELECT TOP 2
    U.Id AS '@UserId',
    U.DisplayName,
    U.UpVotes AS 'Votes/UpVotes',
    U.DownVotes AS 'Votes/DownVotes'
FROM dbo.Users AS U
FOR XML PATH('User'), ROOT('Users');

(2 rows affected)

Total execution time: 00:00:00.023

XML_F52E2B61-18A1-11d1-B105-00805F49916B
"<Users><User UserId=""17""><DisplayName>Nick Berardi</DisplayName><Votes><UpVotes>884</UpVotes><DownVotes>215</DownVotes></Votes></User><User UserId=""19""><DisplayName>Mads Kristiansen</DisplayName><Votes><UpVotes>36</UpVotes><DownVotes>12</DownVotes></Votes></User></Users>"


And now we have this:

<table border="0" cellpadding="0" cellspacing="0" style="border-collapse: separate; width: 600px; box-sizing: content-box !important; border-width: 0px !important; border-style: initial !important; margin: 0px !important; border-radius: 0px !important; background: none !important; float: none !important; height: auto !important; line-height: 1.1em !important; outline: 0px !important; overflow: visible !important; padding: 0px !important; position: static !important; vertical-align: baseline !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important;"><tbody style="box-sizing: content-box !important; border-radius: 0px !important; background: none !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important;"><tr style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-color: initial !important; border-width: 0px !important; border-style: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important;"><td class="gutter" style="box-sizing: content-box !important; border-width: 0px !important; border-style: initial !important; padding: 0px !important; border-radius: 0px !important; background: none !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; color: rgb(175, 175, 175) !important;"><div class="line number1 index0 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">1</div><div class="line number2 index1 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">2</div><div class="line number3 index2 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">3</div><div class="line number4 index3 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">4</div><div class="line number5 index4 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">5</div><div class="line number6 index5 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">6</div><div class="line number7 index6 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">7</div><div class="line number8 index7 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">8</div><div class="line number9 index8 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">9</div><div class="line number10 index9 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">10</div><div class="line number11 index10 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">11</div><div class="line number12 index11 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">12</div><div class="line number13 index12 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">13</div><div class="line number14 index13 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">14</div><div class="line number15 index14 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">15</div><div class="line number16 index15 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">16</div></td><td class="code" style="width: 555.333px; box-sizing: content-box !important; border-width: 0px !important; border-style: initial !important; padding: 0px !important; border-radius: 0px !important; background: none !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important;"><div class="container" style="box-sizing: content-box !important; border-radius: 0px !important; background: none !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px !important; position: relative !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important;"><div class="line number1 index0 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Users</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number2 index1 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">User</code> <code class="xml color1" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: gray !important;">UserId</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">=</code><code class="xml string" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: blue !important;">"17"</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number3 index2 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DisplayName</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;Nick Berardi&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DisplayName</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number4 index3 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Votes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number5 index4 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">UpVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;884&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">UpVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number6 index5 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DownVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;215&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DownVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number7 index6 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Votes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number8 index7 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">User</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number9 index8 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">User</code> <code class="xml color1" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: gray !important;">UserId</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">=</code><code class="xml string" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: blue !important;">"19"</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number10 index9 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DisplayName</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;Mads Kristiansen&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DisplayName</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number11 index10 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Votes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number12 index11 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">UpVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;36&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">UpVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number13 index12 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DownVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;12&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DownVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number14 index13 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Votes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number15 index14 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">User</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number16 index15 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Users</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div></div></td></tr></tbody></table>

## How Do I Add Repeating Sub Elements to XML With SQL Server

Things get a little messier when it comes to adding repeating elements but the approach is relatively simple. An XML element can contain element’s itself so all we need to do is push in a sub query that generates that XML. We’ve already seen how to do this above.  
I’m going to get some details on badges for each user, limited and grouped in this case to keep the results presentable.

Note in the sub query we specify the TYPE directive. This tells the server to keep the results as XML, if we don’t do this then the server assumes you want the results formatted as text, escapes all the special characters and makes a big mess.

In [5]:
SELECT TOP 2
U.Id AS '@UserId',
U.DisplayName,
U.UpVotes AS 'Votes/UpVotes',
U.DownVotes AS 'Votes/DownVotes',
(   SELECT TOP 2
        B.Name, MIN(B.Date) AS FirstAwarded FROM dbo.Badges AS B
    WHERE B.UserId = U.Id
    GROUP BY B.Name
    FOR XML PATH ('Badge'), TYPE
) AS 'Badges'
FROM dbo.Users AS U
FOR XML PATH('User'), ROOT('Users');

(2 rows affected)

Total execution time: 00:00:01.917

XML_F52E2B61-18A1-11d1-B105-00805F49916B
"<Users><User UserId=""17""><DisplayName>Nick Berardi</DisplayName><Votes><UpVotes>884</UpVotes><DownVotes>215</DownVotes></Votes><Badges><Badge><Name>.net</Name><FirstAwarded>2009-02-10T05:11:54.607</FirstAwarded></Badge><Badge><Name>asp.net</Name><FirstAwarded>2009-04-15T03:45:30.720</FirstAwarded></Badge></Badges></User><User UserId=""19""><DisplayName>Mads Kristiansen</DisplayName><Votes><UpVotes>36</UpVotes><DownVotes>12</DownVotes></Votes><Badges><Badge><Name>Beta</Name><FirstAwarded>2008-09-16T00:00:00</FirstAwarded></Badge><Badge><Name>Citizen Patrol</Name><FirstAwarded>2008-09-15T08:55:05.157</FirstAwarded></Badge></Badges></User></Users>"


And now we get this:

<table border="0" cellpadding="0" cellspacing="0" style="border-collapse: separate; width: 604.667px; box-sizing: content-box !important; border-width: 0px !important; border-style: initial !important; margin: 0px !important; border-radius: 0px !important; background: none !important; float: none !important; height: auto !important; line-height: 1.1em !important; outline: 0px !important; overflow: visible !important; padding: 0px !important; position: static !important; vertical-align: baseline !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important;"><tbody style="box-sizing: content-box !important; border-radius: 0px !important; background: none !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important;"><tr style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-color: initial !important; border-width: 0px !important; border-style: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important;"><td class="gutter" style="box-sizing: content-box !important; border-width: 0px !important; border-style: initial !important; padding: 0px !important; border-radius: 0px !important; background: none !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; color: rgb(175, 175, 175) !important;"><div class="line number1 index0 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">1</div><div class="line number2 index1 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">2</div><div class="line number3 index2 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">3</div><div class="line number4 index3 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">4</div><div class="line number5 index4 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">5</div><div class="line number6 index5 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">6</div><div class="line number7 index6 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">7</div><div class="line number8 index7 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">8</div><div class="line number9 index8 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">9</div><div class="line number10 index9 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">10</div><div class="line number11 index10 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">11</div><div class="line number12 index11 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">12</div><div class="line number13 index12 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">13</div><div class="line number14 index13 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">14</div><div class="line number15 index14 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">15</div><div class="line number16 index15 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">16</div><div class="line number17 index16 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">17</div><div class="line number18 index17 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">18</div><div class="line number19 index18 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">19</div><div class="line number20 index19 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">20</div><div class="line number21 index20 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">21</div><div class="line number22 index21 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">22</div><div class="line number23 index22 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">23</div><div class="line number24 index23 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">24</div><div class="line number25 index24 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">25</div><div class="line number26 index25 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">26</div><div class="line number27 index26 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">27</div><div class="line number28 index27 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">28</div><div class="line number29 index28 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">29</div><div class="line number30 index29 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">30</div><div class="line number31 index30 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">31</div><div class="line number32 index31 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">32</div><div class="line number33 index32 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">33</div><div class="line number34 index33 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">34</div><div class="line number35 index34 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">35</div><div class="line number36 index35 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border-width: 0px 3px 0px 0px !important; border-top-style: initial !important; border-right-style: solid !important; border-bottom-style: initial !important; border-left-style: initial !important; border-top-color: initial !important; border-right-color: rgb(108, 226, 108) !important; border-bottom-color: initial !important; border-left-color: initial !important; border-image: initial !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 0.5em 0px 1em !important; position: static !important; text-align: right !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;">36</div></td><td class="code" style="width: 560px; box-sizing: content-box !important; border-width: 0px !important; border-style: initial !important; padding: 0px !important; border-radius: 0px !important; background: none !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important;"><div class="container" style="box-sizing: content-box !important; border-radius: 0px !important; background: none !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px !important; position: relative !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important;"><div class="line number1 index0 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Users</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number2 index1 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">User</code> <code class="xml color1" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: gray !important;">UserId</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">=</code><code class="xml string" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: blue !important;">"17"</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number3 index2 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DisplayName</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;Nick Berardi&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DisplayName</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number4 index3 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Votes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number5 index4 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">UpVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;884&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">UpVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number6 index5 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DownVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;215&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DownVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number7 index6 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Votes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number8 index7 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Badges</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number9 index8 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Badge</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number10 index9 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Name</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;.net&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Name</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number11 index10 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">FirstAwarded</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;2009-02-10T05:11:54.607&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">FirstAwarded</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number12 index11 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Badge</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number13 index12 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Badge</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number14 index13 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Name</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;asp.net&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Name</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number15 index14 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">FirstAwarded</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;2009-04-15T03:45:30.720&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">FirstAwarded</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number16 index15 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Badge</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number17 index16 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Badges</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number18 index17 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">User</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number19 index18 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">User</code> <code class="xml color1" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: gray !important;">UserId</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">=</code><code class="xml string" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: blue !important;">"19"</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number20 index19 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DisplayName</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;Mads Kristiansen&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DisplayName</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number21 index20 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Votes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number22 index21 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">UpVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;36&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">UpVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number23 index22 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DownVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;12&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">DownVotes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number24 index23 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Votes</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number25 index24 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Badges</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number26 index25 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Badge</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number27 index26 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Name</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;Beta&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Name</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number28 index27 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">FirstAwarded</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;2008-09-16T00:00:00&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">FirstAwarded</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number29 index28 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Badge</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number30 index29 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Badge</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number31 index30 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Name</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;Citizen Patrol&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Name</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number32 index31 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">FirstAwarded</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;2008-09-15T08:55:05.157&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">FirstAwarded</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number33 index32 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Badge</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number34 index33 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Badges</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number35 index34 alt2" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml spaces" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important;">&nbsp;&nbsp;</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">User</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div><div class="line number36 index35 alt1" style="box-sizing: content-box !important; border-radius: 0px !important; background-image: none !important; background-position: initial !important; background-size: initial !important; background-repeat: initial !important; background-attachment: initial !important; background-origin: initial !important; background-clip: initial !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; padding: 0px 1em !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-size: 1em !important; direction: ltr !important; box-shadow: none !important; white-space: pre !important;"><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&lt;/</code><code class="xml keyword" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; font-weight: bold !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: rgb(0, 102, 153) !important;">Users</code><code class="xml plain" style="box-sizing: content-box !important; font-size: 1em !important; font-family: Monaco, Consolas, &quot;Bitstream Vera Sans Mono&quot;, &quot;Courier New&quot;, Courier, monospace !important; background: none !important; padding: 0px !important; border-radius: 0px !important; border: 0px !important; float: none !important; height: auto !important; line-height: 1.1em !important; margin: 0px !important; outline: 0px !important; overflow: visible !important; position: static !important; vertical-align: baseline !important; width: auto !important; direction: ltr !important; box-shadow: none !important; display: inline !important; color: black !important;">&gt;</code></div></div></td></tr></tbody></table>

I’m not a fan of sub queries directly in the SELECT statement of a query and possibly a little too keen on using APPLY so I’d probably re-write the above like the following.  
The ‘\*’ on the field name is a place holder for nothing so we don’t force in another element level. Equally we could exclude the Root of ‘users’ and use that as the field name and get the same result.

In [6]:
SELECT TOP 2
U.Id AS '@UserId',
U.DisplayName,
U.UpVotes AS 'Votes/UpVotes',
U.DownVotes AS 'Votes/DownVotes',
B.x AS '*'
FROM dbo.Users AS U
OUTER APPLY
(
    SELECT B.Name, MIN(B.Date) AS FirstAwarded FROM dbo.Badges AS B
    WHERE B.UserId = U.Id
    GROUP BY B.Name
    FOR XML PATH ('Badge'), ROOT('Badges'), TYPE
) B(x)
FOR XML PATH('User'), ROOT('Users')
;

(2 rows affected)

Total execution time: 00:00:00.743

XML_F52E2B61-18A1-11d1-B105-00805F49916B
"<Users><User UserId=""17""><DisplayName>Nick Berardi</DisplayName><Votes><UpVotes>884</UpVotes><DownVotes>215</DownVotes></Votes><Badges><Badge><Name>.net</Name><FirstAwarded>2009-02-10T05:11:54.607</FirstAwarded></Badge><Badge><Name>asp.net</Name><FirstAwarded>2009-04-15T03:45:30.720</FirstAwarded></Badge><Badge><Name>asp.net-mvc</Name><FirstAwarded>2010-01-10T21:00:26.830</FirstAwarded></Badge><Badge><Name>Autobiographer</Name><FirstAwarded>2008-09-15T08:55:05.250</FirstAwarded></Badge><Badge><Name>Benefactor</Name><FirstAwarded>2010-07-09T09:55:47.577</FirstAwarded></Badge><Badge><Name>Beta</Name><FirstAwarded>2008-09-16T00:00:00</FirstAwarded></Badge><Badge><Name>c#</Name><FirstAwarded>2009-01-30T18:44:35.257</FirstAwarded></Badge><Badge><Name>Caucus</Name><FirstAwarded>2012-06-08T23:11:15.383</FirstAwarded></Badge><Badge><Name>Citizen Patrol</Name><FirstAwarded>2008-09-15T08:55:05.157</FirstAwarded></Badge><Badge><Name>Civic Duty</Name><FirstAwarded>2008-12-05T18:42:34.357</FirstAwarded></Badge><Badge><Name>Cleanup</Name><FirstAwarded>2008-09-15T08:55:04.470</FirstAwarded></Badge><Badge><Name>Commentator</Name><FirstAwarded>2008-10-07T11:27:34.803</FirstAwarded></Badge><Badge><Name>Constituent</Name><FirstAwarded>2012-06-08T23:13:26.880</FirstAwarded></Badge><Badge><Name>Critic</Name><FirstAwarded>2008-09-15T08:55:04.970</FirstAwarded></Badge><Badge><Name>Custodian</Name><FirstAwarded>2012-09-21T23:36:58.153</FirstAwarded></Badge><Badge><Name>Deputy</Name><FirstAwarded>2011-06-21T00:45:57.857</FirstAwarded></Badge><Badge><Name>Disciplined</Name><FirstAwarded>2008-10-30T13:22:34.447</FirstAwarded></Badge><Badge><Name>Editor</Name><FirstAwarded>2008-09-15T08:55:04.360</FirstAwarded></Badge><Badge><Name>Enlightened</Name><FirstAwarded>2009-01-03T15:37:36.213</FirstAwarded></Badge><Badge><Name>Enthusiast</Name><FirstAwarded>2011-06-07T15:01:21.220</FirstAwarded></Badge><Badge><Name>Excavator</Name><FirstAwarded>2011-08-15T10:46:02.190</FirstAwarded></Badge><Badge><Name>Famous Question</Name><FirstAw"


And that should cover 99% of needs when it comes to building XML. You might need to take the sub query concept a little further, potentially needing sub queries in your sub queries but with the above you can put together pretty much anything.

Querying XML In SQL Server Series

- [Part 1 – Introduction](https://barneylawrence.com/2021/03/15/querying-xml-in-sql-server-part-1-introduction/)
- [Part 2 – Simple Paths and Attributes With the Value Method](https://barneylawrence.com/2021/03/23/querying-xml-in-sql-server-part-2-simple-paths-and-attributes-with-the-value-method/)
- [Part 3 – Handling Repeating Regions With the Nodes Method](https://barneylawrence.com/2021/03/30/querying-xml-in-sql-server-part-3-handling-repeating-regions-with-the-nodes-method/)
- [Part 4 – Filtering With The Exist Method](https://barneylawrence.com/2021/04/07/querying-xml-in-sql-server-part-4-filtering-with-the-exist-method/)
- [Part 5 – Introduction to the Query Method and XQuery](https://barneylawrence.com/2021/04/21/querying-xml-in-sql-server-part-5-an-introduction-to-the-query-method-and-xquery/)
- [Part 6 – Sequencing Elements and Finding Positions](https://barneylawrence.com/2021/04/28/querying-xml-in-sql-server-part-6-sequencing-elements-and-finding-positions/)
- [Part 7 – Creating XML](https://barneylawrence.com/2021/05/06/querying-xml-in-sql-server-part-7-creating-xml/)

[Posts in SQL Notebook form](https://github.com/BarneyLawrence/XML-Queries-In-SQL-Server/tree/main/Notebooks)